In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from sqlalchemy import create_engine

In [10]:
fanfics = os.listdir('Fanfic_HTML')

#remove the below code if the .DS_Store doesn't exist
remove_fanfics = ['.DS_Store']
for item in remove_fanfics:
    fanfics.remove(item)

for index, item in enumerate(fanfics):
    fanfics[index] = 'Fanfic_HTML/' + item

In [11]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractRelationships(classTags):
    #code to extract relationships
    i = 0
    j = 0
    story_relationships = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Relationship:":
            j = 2 + i
        elif i == j:
            raw_relationships = element.find_all('a')
            for obj in raw_relationships:
                story_relationships += str(obj.string) + ', '
            story_relationships = story_relationships[:-2]
            
    return story_relationships

def extractCharacters(classTags):
    #code to extract characters
    i = 0
    j = 0
    characters = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Character:":
            j = 2 + i
        elif i == j:
            raw_characters = element.find_all('a')
            for obj in raw_characters:
                characters += str(obj.string) + ', '
            characters = characters[:-2]
            
    return characters

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

def extractSeries(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Series:":
            j = 2 + i
        elif i == j:
            series = str(element.find('a').string)

    return series

def extractCollections(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    collections = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Collections:":
            j = 2 + i
        elif i == j:
            collections = str(element.find('a').string)

    return collections

def extractPubStats(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Stats:":
            j = 2 + i
        elif i == j:
            pub_stat = str(element.string).strip()
            stat_length = len(pub_stat)
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[stat_length-4:stat_length+1]
            if " " not in story_word_count:
                continue
            else: 
                story_word_count = story_word_count[1:]
    
    return story_publish_date, story_word_count

In [12]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = np.nan
    story_character = np.nan
    relationship = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    series = np.nan
    collections = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            fandom_text = extractFandoms(tagsClass)
        
        elif str(element.string) == "Relationship:":
            relationship = extractRelationships(tagsClass)
            
        elif str(element.string) == "Character:":
            story_character = extractCharacters(tagsClass)
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Series:":
            series = extractSeries(tagsClass)
            
        elif str(element.string) == "Collections:":
            collections = extractCollections(tagsClass)
            
        elif str(element.string) == "Stats:":
            pubDate, wordCount = extractPubStats(tagsClass)
            
        i += 1
    
    local_tag_dict = {'rating' : rating,
                'archiveWarnings': archiveWarning,
                'category' : category,
                'fandom' : fandom_text,
                'relationships' : relationship,
                'character': story_character,
                'additionalTags': addTags,  
                'series': series, 
                'collections': collections,
                'pub_date' : pubDate, 
                'word_count' : wordCount
               }
        
    return local_tag_dict

In [13]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author

def extractTitle(soup_obj):
    #code to extract title
    i = 0
    for element in soup_obj.find(class_="message"):
        if i == 1:
            title = element.string
        elif i > 1:
            break
        i += 1
        
    return title
    
def extractStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = """"""
    for element in soup_obj.find(id="chapters"):
        if i == 3:
            if "<span>" in str(soup_obj):
                story_spantag_text = element.find_all('span')
                for obj in story_spantag_text:
                    if obj.string == None:
                        continue
                    else:
                        story_text += str(obj.string) + " "
                        
            elif "<div>" in str(soup_obj):
                story_divtag_text = element.find('div')
                story_ptag_text = story_divtag_text.find_all('p')
                for obj in story_ptag_text:
                    features = obj.contents
                    for feature in features:
                        if "br" in str(feature):
                            continue
                        else:
                            story_text += str(feature) + " " 
                            story_text = story_text.replace("<i>", "")
                            story_text = story_text.replace("</i>", "")
                        
            else:
                story_ptag_text = element.find_all('p')
                if len(story_ptag_text) == 1:
                    story_text = str(element.find('p'))
                    story_text = story_text.replace("<p>", "")
                    story_text = story_text.replace("</p>", "")
                    story_text = story_text.replace("<br/>", "")
                else:
                    for obj in story_ptag_text:
                        if obj.string == None:
                            continue
                        else:
                            story_text += str(obj.string) + " "
                       
        elif i > 3:
            break
            
        i += 1
        
    return story_text


def extractStoryChapters(soup_obj):
    #code to extract title and story text
    i = 0
    j = 0
    story_chapter_text = ""
    chapters = soup_obj.find(id="chapters")
    for element in soup_obj.find(id="chapters"):
        i += 1
        if str(element.string) == "chapter content":
            j = 2 + i
        elif str(element.string) == "/chapter content":
            j = 2 + i
        elif i == j:
            if 'blockquote class="userstuff"' in str(chapters.find(class_="userstuff")):
                k = 0
                m = 0
                for element in chapters:
                    k += 1
                    if str(element.string) == "chapter content":
                        m = 2 + k
                    elif k == m:
                        chapter_ptag_text = element.find_all('p')
                        
            elif "<span>" in str(chapters.find(class_="userstuff")):
                n = 0
                o = 0
                for element in chapters:
                    n += 1
                    if str(element.string) == "chapter content":
                        o = 2 + n
                    elif n == o:
                        chapter_ptag_text = element.find_all('span')
                        
            else:
                userStuff = chapters.find(class_="userstuff")
                chapter_ptag_text = userStuff.find_all('p')
                
            for obj in chapter_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_chapter_text += str(obj.string) + " "
                    
    return story_chapter_text

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    '''
    INPUT:
    filelist - (list) list of HTML files to process
   
    OUTPUT:
    ao3_dict - (dictionary) dictionary of processed AO3 files 
    
    '''
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        tag_dict = extractTags(soup)
        author = extractAuthor(soup)
        title = extractTitle(soup)
        if "chapter content" in str(soup):
            story_text = extractStoryChapters(soup)
        else:
            story_text = extractStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'url': url,
                     }
        story_dict.update(tag_dict)
        
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [15]:
ao3_dictionary = createAO3dict(fanfics)

#convert dictionary to dataframe
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")

#convert word_count column to int32 datatype
#ao3_df.word_count.astype('int32')

In [17]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")

In [18]:
ao3_df.to_csv('inprocess.csv', index=False)

In [ ]:
#grab all the blank story_text
blankStory_df = ao3_df[ao3_df["story_text"] == '']

#choose to drop blank stories or reading the in the stories manually

#for any stories that are acutally blank and need to be dropped
#ao3_df = ao3_df.drop(blankStory_df.index)

In [11]:
#check that all blank stories have been dropped
blankStory_df = ao3_df[ao3_df["story_text"] == '']
blankStory_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count


In [12]:
def listifyTags(column):
    #create tag lists for manual processing
    unique_tags_list = ao3_df[column].unique().tolist()

    tag_list = []

    for i in unique_tags_list:
        tag = str(i).split(", ")
        tag_list.append(tag)

    flattened_tag_list = [item for sublist in tag_list for item in sublist]

    flattened_tag_list = list(set(flattened_tag_list))

    sorted_flattened_tag_list = sorted(flattened_tag_list)

    tag_df = pd.DataFrame(sorted_flattened_tag_list,columns=['tags'])
    tag_df.to_csv('UniqueList_{}.csv'.format(column), index=False)

In [13]:
listifyTags('additionalTags')
listifyTags('fandom')

In [16]:
#read in manually processed tag lists and 
fandomsDF = pd.read_csv("tags_CSV/fandomTags.csv", header=None)
fandomGenreDF = pd.read_csv("tags_CSV/fandomGenres.csv", header=None)
tagsDF = pd.read_csv("tags_CSV/allTags.csv", header=None)
inProcessAO3_df = pd.read_csv("inProcessAO3data.csv")

#drop header rows
fandomsDF = fandomsDF.drop(0)
fandomGenreDF = fandomGenreDF.drop(0)

In [17]:
def createTagDict(df):
    '''
    INPUT:
    df - (pandas dataframe) df as defined at the top of the notebook 
    
    OUTPUT:
    tagDictionary - (dictionary) dictionary of keys to map to values 
    
    '''
    tagIndicesList = list(df[0])
    tagDictionary = dict()
    i = 0
    for variableTag in tagIndicesList:
        for element in df.iloc[i]:
            if str(element) != 'nan':
                #append tags to value for tags key
                if variableTag in tagDictionary.keys():
                    tagDictionary[variableTag].append(element)
                #create key and initialize an empty list to append values to, skippings first value because that's the 'tags' value
                elif variableTag not in tagDictionary.keys():
                    tagDictionary[variableTag] = list()
        i += 1

    return tagDictionary

In [18]:
#create tag dictionaries from tag groupings data
tagsDictionary = createTagDict(tagsDF)
fandomDictionary = createTagDict(fandomsDF)
fandomGenreDictionary = createTagDict(fandomGenreDF)

In [19]:
#listify tag and fandom columns
inProcessAO3_df['consolidatedFandom'] = inProcessAO3_df.fandom.str.split(",")
inProcessAO3_df['tags'] = inProcessAO3_df.additionalTags.str.split(",")

In [20]:
def mapDictKeys2Column(column, dictionary):
    '''
    INPUT:
    column - (pandas dataframe column) column with values that need to be updated
    dictionary - (dictionary) passed dictionary to map column values to dict keys
    
    OUTPUT:
    column, column1 - (pandas dataframe column) columns with remapped values
    
    '''
    for i, tagsList in enumerate(column):
        if type(tagsList) == type(np.nan):
            continue
        else:
            strippedTagsList = [s.strip() for s in tagsList]
            for key, values in dictionary.items():
                for j, tag in enumerate(strippedTagsList):
                    if tag in values:
                        for value in values:
                            if tag != value:
                                continue
                            else:
                                strippedTagsList[j] = key

        column[i] = list(set(strippedTagsList))
        column1 = column
    
    return column, column1

def mapDictValues2Column(column, dictionary):
    '''
    INPUT:
    column - (pandas dataframe column) column with values that need to be updated
    dictionary - (dictionary) passed dictionary to map column values to dict keys
    
    OUTPUT:
    column, column1 - (pandas dataframe column) columns with remapped values, ready to one-hot encode 
    
    '''
    
    for i, tagsList in enumerate(column):
        if type(tagsList) == type(np.nan):
            continue
        else:
            strippedTagsList = [s.strip() for s in tagsList]
            for key, values in dictionary.items():
                for j, tag in enumerate(strippedTagsList):
                    if tag != key:
                        continue
                    else:
                        strippedTagsList[j] = values

        flattened_strippedTagsList = [item for sublist in strippedTagsList for item in sublist]

        column[i] = flattened_strippedTagsList
        column1 = column
    
    return column, column1

In [21]:
#map dictionary values to processed data columns
inProcessAO3_df['consolidatedFandom'], inProcessAO3_df['consolidatedFandom1'] = mapDictKeys2Column(inProcessAO3_df['consolidatedFandom'], fandomDictionary)
inProcessAO3_df['fandomGenres'] = inProcessAO3_df['consolidatedFandom']
inProcessAO3_df['tags'], inProcessAO3_df['tags1'] = mapDictKeys2Column(inProcessAO3_df['tags'], tagsDictionary)
inProcessAO3_df['tags'], inProcessAO3_df['tags1'] = mapDictKeys2Column(inProcessAO3_df['tags'], fandomDictionary)
inProcessAO3_df['fandomGenres'], inProcessAO3_df['fandomGenres1'] = mapDictValues2Column(inProcessAO3_df['fandomGenres'], fandomGenreDictionary)

#grab columns that will need to be dropped later
intDropList = list(inProcessAO3_df.columns)

<ipython-input-20-71a7f2d35016>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i] = list(set(strippedTagsList))
<ipython-input-20-71a7f2d35016>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[i] = flattened_strippedTagsList


In [22]:
def oneHotEncode(df, colName):
    #one-hot encode passed column
    df = df.explode(colName + str(1))
    dummies = pd.get_dummies(df[colName + str(1)], prefix=colName)
    df = pd.concat([df, dummies], axis=1)
    
    return df

In [23]:
#one hot encode data and finish processing data to have a machine learning ready dataset
inProcessAO3_df = oneHotEncode(inProcessAO3_df, "tags")
inProcessAO3_df = oneHotEncode(inProcessAO3_df, "fandomGenres")
inProcessAO3_df = oneHotEncode(inProcessAO3_df, "consolidatedFandom")

#convert 0 to NaNs so groupby method can be used to implode dataset
inProcessAO3_df = inProcessAO3_df.replace(0, np.nan)

# 'implode' dataset and remove index column
inProcessAO3_df = inProcessAO3_df.groupby(list(inProcessAO3_df.index)).first().reset_index()
inProcessAO3_df = inProcessAO3_df.drop(columns=['index'])

#convert NaNs in one-hot encoded columns back to 0
intInProcess_AO3_df = inProcessAO3_df.drop(columns=intDropList)
intInProcess_AO3_df = intInProcess_AO3_df.replace(np.nan, 0)
strDropList = list(intInProcess_AO3_df.columns)
strInProcess_AO3_df = inProcessAO3_df.drop(columns=strDropList)
intInProcess_AO3_df = intInProcess_AO3_df.astype(int)

processedAO3df = pd.merge(strInProcess_AO3_df, intInProcess_AO3_df, on=intInProcess_AO3_df.index)

#create columns that can be manipulated and dropped while keeping new columns
processedAO3df = processedAO3df.drop(columns=["key_0", 'tags_disregardTags',  "tags1", "consolidatedFandom1", "fandomGenres1"])

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,...,consolidatedFandom_theBellesTags,consolidatedFandom_thePowerTags,consolidatedFandom_trumanShowTags,consolidatedFandom_umbrellaAcademyTags,consolidatedFandom_underworldTags,consolidatedFandom_vagrantQueenTags,consolidatedFandom_wayfarersSeriesTags,consolidatedFandom_whiteCollarTags,consolidatedFandom_worstWitchTags,consolidatedFandom_wrinkleInTimeTags
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,Partnership,roguefaerie,Sometimes Winchester thought about what other ...,http://archiveofourown.org/works/21432643,General Audiences,No Archive Warnings Apply,Other,"Supernatural, American Gods - Neil Gaiman",Shadow Moon (American Gods)/Dean Winchester,"Dean Winchester, Shadow Moon (American Gods)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,Partnership,roguefaerie,Sometimes Winchester thought about what other ...,http://archiveofourown.org/works/21432643,General Audiences,No Archive Warnings Apply,Other,"Supernatural, American Gods - Neil Gaiman",Shadow Moon (American Gods)/Dean Winchester,"Dean Winchester, Shadow Moon (American Gods)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,Partnership,roguefaerie,Sometimes Winchester thought about what other ...,http://archiveofourown.org/works/21432643,General Audiences,No Archive Warnings Apply,Other,"Supernatural, American Gods - Neil Gaiman",Shadow Moon (American Gods)/Dean Winchester,"Dean Winchester, Shadow Moon (American Gods)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,Partnership,roguefaerie,Sometimes Winchester thought about what other ...,http://archiveofourown.org/works/21432643,General Audiences,No Archive Warnings Apply,Other,"Supernatural, American Gods - Neil Gaiman",Shadow Moon (American Gods)/Dean Winchester,"Dean Winchester, Shadow Moon (American Gods)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def listToString(df, colName):
    #convert lists in passed columns to strings to store data in SQLite database
    for index, element in enumerate(df[colName]):
        if type(element) == type(np.nan):
                continue
        elif len(element) == 1:
            df[colName][index] = element[0]
        else:
            newElement = ", ".join([str(cell) for cell in element])
            df[colName][index] = newElement

    return df

In [ ]:
#process data to be able to store dataset in SQLite database
processedAO3df = listToString(processedAO3df, "consolidatedFandom")
processedAO3df = listToString(processedAO3df, "fandomGenres")
processedAO3df = listToString(processedAO3df, "tags")

In [ ]:
#create SQLite database and store dataset as tables in database
engine = create_engine('sqlite:///fictionTaggingEngine.db')
tagsDF.to_sql("tags", engine, index=False)
fandomsDF.to_sql("fandoms", engine, index=False)
fandomGenreDF.to_sql("fandomGenre", engine, index=False)
processedAO3df.to_sql("processedAO3data", engine, index=False)